In [ ]:
import os
os.chdir('/content/drive/My Drive/종합설계')

In [ ]:
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Conv3D, BatchNormalization, ReLU, Add, MaxPool3D, GlobalAveragePooling3D, Concatenate, Dropout, Dense, Lambda, Input
from tensorflow.keras.models import Model
from tensorflow.keras import Sequential
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.regularizers import l2
from myslowfast.non_local import non_local_block

In [ ]:
def Conv_BN_ReLU(planes, kernel_size, name, strides=(1, 1, 1), padding='same', use_bias=False, kernel_regularizer_rate=0.001):
  if K.image_data_format() == 'channels_last':
    bn_axis = 4
  else:
    bn_axis = 1
  return Sequential([
        Conv3D(planes, kernel_size, strides=strides, padding=padding, use_bias=use_bias, kernel_regularizer=l2(kernel_regularizer_rate)),
        BatchNormalization(axis=bn_axis),
        ReLU()])
  
def bottleneck(inputs, planes, name, stage, stride=1, downsample=None, head_conv=1, use_bias=False, kernel_regularizer_rate = 0.001):
    if K.image_data_format() == 'channels_last':
      bn_axis = 4
    else:
      bn_axis = 1
    x=inputs
    residual = inputs
    if head_conv == 1:
        x = Conv_BN_ReLU(planes, kernel_size=1, use_bias=use_bias, name = name+'_'+str(stage)+'_'+'first')(x)
    elif head_conv == 3:
        x = Conv_BN_ReLU(planes, kernel_size=(3, 1, 1), use_bias=use_bias, name = name+'_'+str(stage)+'_'+'first')(x)
    else:
        raise ValueError('Unsupported head_conv!!!')
    x = Conv_BN_ReLU(planes = planes, kernel_size=(1, 3, 3), strides=(1, stride, stride), use_bias=use_bias, name = name+'_'+str(stage)+'_'+'second')(x)
    x = Conv3D(planes*4, kernel_size=1, use_bias=use_bias, kernel_regularizer=l2(kernel_regularizer_rate), name = name+'_'+str(stage)+'_'+'third')(x)
    x = BatchNormalization(axis=bn_axis)(x)
    if stage==0:
        downsample = Sequential([
            Conv3D(planes*4, kernel_size=1, strides=(1, stride, stride), use_bias=False),
            BatchNormalization(axis = bn_axis)
        ])
        residual = downsample(residual)
    x = Add()([x, residual])
    x = ReLU()(x)
    return x

def datalayer(x, stride):
    return x[:, ::stride, :, :, :]

def SlowFast_body(input_shape, num_classes, layers=(3,4,6,3), tau = 16, alpha = 8, beta = 1/8, dropout=0.5):
    inputs = Input(shape=(input_shape))
    inputs_fast = Lambda(datalayer, name='data_fast', arguments={'stride':int(tau/alpha)})(inputs)
    print(inputs_fast.shape)
    inputs_slow = Lambda(datalayer, name='data_slow', arguments={'stride':tau})(inputs)
    print(inputs_slow.shape)
    fast, lateral = Fast_body(inputs_fast, layers, alpha=alpha, beta=beta)
    slow = Slow_body(inputs_slow, lateral, layers)
    x = Concatenate()([slow, fast])
    x = Dropout(dropout)(x)
    out = Dense(num_classes, activation='softmax')(x)
    return Model(inputs, out)

def make_layer_fast(inputs, planes, blocks, name, stride=1, head_conv=1):
    
    '''
    if stride!=1 :
        downsample = Sequential([
            Conv3D(planes*4, kernel_size=1, strides=(1, stride, stride), use_bias=False),
            BatchNormalization(axis = bn_axis)
        ],name=name)
    '''
    x = bottleneck(inputs = inputs, planes = planes, stride = stride, head_conv=head_conv,name=name, stage=0)
    for i in range(1, blocks):
        x = bottleneck(inputs = x, planes = planes, head_conv=head_conv, name = name, stage = i)
    return x

def Fast_body(x, layers, alpha, beta):
    fast_inplanes = 8
    #res2 부터 필터
    slow_filters = [64,128,256,512] 
    #8,16,32,64
    fast_filters = [slow_filters[0]*beta, slow_filters[1]*beta, slow_filters[2]*beta,slow_filters[3]*beta]
    
    lateral = []
    x = Conv_BN_ReLU(planes = 8, kernel_size=(5, 7, 7), strides=(1, 2, 2), name = 'fastconv1')(x)
    x = MaxPool3D(pool_size=(1, 3, 3), strides=(1, 2, 2), padding='same', name='fastpool1')(x)
    lateral_p1 = Conv3D(8*2, kernel_size=(5, 1, 1), strides=(alpha, 1, 1), padding='same', use_bias=False)(x)
    lateral.append(lateral_p1)
    
    x = make_layer_fast(inputs = x, planes= fast_filters[0], blocks=layers[0], head_conv=3, name='fastres2')
    lateral_res2 = Conv3D(slow_filters[0], kernel_size=(5, 1, 1), strides=(alpha, 1, 1), padding='same', use_bias=False, name='fastres2_leteral')(x)
    lateral.append(lateral_res2)
    
    x = make_layer_fast(inputs = x, planes = fast_filters[1], blocks=layers[1], stride=2, head_conv=3, name='fastres3')
    lateral_res3 = Conv3D(slow_filters[1], kernel_size=(5, 1, 1), strides=(alpha, 1, 1), padding='same', use_bias=False, name='fastres3_leteral')(x)
    lateral.append(lateral_res3)

    x = non_local_block(x,compression=2, mode='dot')
    
    x = make_layer_fast(inputs = x, planes = fast_filters[2], blocks=layers[2], stride=2, head_conv=3, name='fastres4')
    lateral_res4 = Conv3D(slow_filters[2], kernel_size=(5, 1, 1), strides=(alpha, 1, 1), padding='same', use_bias=False, name='fastres4_leteral')(x)
    lateral.append(lateral_res4)

    x = make_layer_fast(inputs = x, planes = fast_filters[3], blocks=layers[3], stride=2, head_conv=3, name='fastres5')
    x = GlobalAveragePooling3D()(x)
    return x, lateral

def make_layer_slow(inputs, planes, blocks, name, stride=1, head_conv=1):
    
    '''
    if stride!=1:
        downsample = Sequential([
            Conv3D(planes*4, kernel_size=1, strides = (1, stride, stride), use_bias=False),
            BatchNormalization()
        ])
    '''
    x = bottleneck(inputs = inputs, planes = planes, stride = stride, head_conv=head_conv, name = name, stage = 0)
    for i in range(1, blocks):
        x = bottleneck(inputs = x, planes = planes, head_conv=head_conv, name = name, stage=i)
    return x

def Slow_body(x, lateral, layers):
    slow_filters = [64,128,256,512] 
    x = Conv_BN_ReLU(64, kernel_size=(1, 7, 7), strides=(1, 2, 2), name='slowconv1')(x)
    x = MaxPool3D(pool_size=(1, 3, 3), strides=(1, 2, 2), padding='same', name='slowpool1')(x)
    x = Concatenate()([x, lateral[0]])

    x = make_layer_slow(inputs = x, planes = slow_filters[0],  blocks = layers[0], head_conv=1, name='slowres2')
    x = Concatenate()([x, lateral[1]])

    x = make_layer_slow(inputs = x, planes = slow_filters[1],  blocks = layers[1], stride=2, head_conv=1, name='slowres3')
    x = Concatenate()([x, lateral[2]])

    x = non_local_block(x,compression=2, mode='dot')
    x= make_layer_slow(inputs = x, planes = slow_filters[2],  blocks = layers[2], stride=2, head_conv=3, name='slowres4')
    x = Concatenate()([x, lateral[3]])

    x = make_layer_slow(inputs = x, planes = slow_filters[3], blocks = layers[3], stride=2, head_conv=3, name='slowres5')
    x = GlobalAveragePooling3D()(x)
    return x


In [ ]:
model = SlowFast_body(input_shape=(64,224,224,3), num_classes=10)

In [ ]:
model.summary()

In [ ]:
for layer in model.layers:
  print(layer.output_shape, layer.name)